In [9]:
import csv
from selenium import webdriver
# import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [10]:
driver_path = "./chromedriver.exe"

chrome_options = Options()

service = Service(driver_path)

driver = webdriver.Chrome(service=service, options=chrome_options)

NoSuchDriverException: Message: Unable to obtain ./chromedriver.exe using Selenium Manager; Message: Unsuccessful command executed: c:\Users\Fame\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\common\windows\selenium-manager.exe --browser chrome --output json.
The chromedriver version cannot be discovered
; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location
